In [7]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import glob
import os

In [8]:
all_data=glob.glob('/Users/nikkibytes/Documents/testing/sub*')
all_data
for sub in all_data:
    print("SUBJECT: ", sub)
    for dir in glob.glob(os.path.join(sub,'fmriprep/sub-001/func')):#path to the functional, skull stripped data
            print("DIRECTORY: ", dir)#not needed but i get crazy 
           

SUBJECT:  /Users/nikkibytes/Documents/testing/sub-001
DIRECTORY:  /Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func


In [ ]:
def preproc(DATA):
    print("Starting motion correction ")
    basedir='/Users/nikkibytes/Documents/testing'
    writedir='/Users/nikkibytes/Documents/testing/sub-001/fmriprep/sub-001/func/motion_assessment'
    
    datestamp=datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S")
    outhtml = os.path.join(writedir,'bold_motion_QA_test_%s.html'%(datestamp))
    out_bad_bold_list = os.path.join(writedir,'testing_%s.txt'%(datestamp))
    print("OUTPUT HTML: ", outhtml )
    for sub in DATA:
        print("SUBJECT: ", sub)